<a href="https://colab.research.google.com/github/All4Nothing/hg-mldl/blob/main/5-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

## 정형 데이터와 비정형 데이터

**정형 데이터 structured data** 어떤 구조로 되어 있는 데이터  
**비정형 데이터** 구조가 없는 데이터

**앙상블 학습 ensemble learning** 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

## 랜덤 포레스트

**랜덤 포레스트 Random Forest**는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 **숲**을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

**부트스트랩 샘플 bootstrap sample** 훈련 데이터에서 랜덤하게 샘플을 복원 추출하여 만든 훈련 데이터  
기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다

분류 모델인 `RandomForestClassifier`는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다  
회귀 모델인 `RandomForestRegressor`는 전체 특성을 사용한다

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

`RandomForestClassifier`는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하는 것이 좋다.

`return_train_score = True` : 검증 전수뿐만 아닌 훈련 세트에 대한 점수도 같이 반환

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다

In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


**OOB out of bag** 샘플 : 부트스트랩 샘플에 포함되지 않고 남는 샘플  
`RandomForestClassifier` 클래스의 `oob_score=True`로 지정하여 OOB 점수 얻을 수 있다.

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)  
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

**엑스트라 트리 Extra Tree**는 기본적으로 100개의 결정 트리를 훈련합니다.  
랜덤 포레스트와 매우 비슷하며, 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용한다  
랜덤 포레스트와는 달리 부트스트랩 샘플을 사용하지 않는다(전체 훈련 세트를 사용한다)  
노들르 분할할 때 무작위로 분할한다(= `DecisionTreeClassifier` `splitter=random`)

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


특성을 무작위로 분할하여 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있고, 계산 속도가 빠르다

In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


엑스트라 트리의 회귀 버전은 `ExtraTreesRegressor`

## 그레이디언트 부스팅

**그레이디언트 부스팅 gradient boosting**은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법

사이킷런의 `GradientBoostingClassifier`는 기본적으로 깊이가 3인 결정 트리를 100개 사용한다. 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 성능을 기대할 수 있다.

경사 하강법을 사용하여 트리를 앙상블에 추가한다  
분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용한다

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 강해, 학습률을 증가시키고 트리의 개수를 늘리면 성능이 향상될 수 있다.

In [10]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [12]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있다. 하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느리다.  
`GradientBoostingClassifier`에는 n_jobs 매개변수가 없다. 

그레이디언트 부스팅의 회귀 버전은 `GradientBoostingRegressor`

## 히스토그램 기반 그레이디언트 부스팅

**히스토그램 기반 그레이디언트 부스팅 Histogram-based Gradient Boosting**은 입력 특성을 256개의 구간으로 나눈다.  
입력에 누락된 특성이 있더라도 따로 전처리할 필요가 없다.

`HistGradientBoostingClassifier`는 트리의 개수를 지정하는데 `n_estimators` 대신 부스팅 반복 횟수를 지정하는 `max_iter`를 사용한다.

In [14]:
from sklearn.experimental import enable_hist_gradient_boosting
import sklearn.ensemble 

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [15]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [16]:
hgb.score(test_input, test_target)

0.8723076923076923

히스토그램 기반 그레이디언트 부스팅의 회귀 버전은 `HistGradientBoostingRegressor`

`XGBoost` 히스토그램 기반 그레이디언트 부스팅 알고리즘. `cross_validate()` 함수와 함께 사용 가능

In [17]:
from pandas.core.common import random_state
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


`LightGBM`

In [18]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
